In [22]:
%pip install mosestokenizer

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49170 sha256=4436a2bd6ec678803ed294649ad00a41f1219831ef014effacb02c2c082d2a9a
  Stored in directory: /root/.cache/pip/wheels/80/d8/15/4c5ebbe883513f003cb055a0369c77c9df857023a706f39e70
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=4bf5dcca589f35ddc7aaa8a634209aee135a968133b84bf43b77cef1776adbd3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3338 sha256=034d7054ec77273cf09ae10573b36762cd0dcb2448c729c09e22a4ea5a8b88ac
  Stored in directory: /root/.cache/pip/wheels/e1/af/b1/99b57a06dda78fdcee86d2e22c64743f3b8df8f31cfc04baf7
  Created wheel for uctools: filenam

In [1]:
HTML = "https://www.gutenberg.org/cache/epub/2814/pg2814-images.html"

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
dubliners = requests.get(HTML)

In [4]:
assert dubliners.status_code == 200

In [5]:
soup = BeautifulSoup(dubliners.text, 'html.parser')

In [7]:
body = soup.find("body")

In [20]:
text = []
for chapter in body.findAll("div", {"class": "chapter"}):
    for element in chapter.findChildren():
        if element.name == "section" and element.get("id") in ["pg-header", "pg-footer"]:
            continue
        stripped = element.text.strip()
        if stripped != "":
            text.append(stripped)

In [23]:
from mosestokenizer import MosesSentenceSplitter

In [33]:
sents = []
try:
    with MosesSentenceSplitter('en') as splitsents:
        for para in text:
          if para == "":
              continue
          sents += splitsents([para.replace("\r\n", " ")])
except Exception as ex:
    print(ex, para)

In [75]:
import re
def cleaner(text):
    text = text.replace("“", "")
    text = text.replace("”", "")
    text = text.replace("’ ", " ")
    text = text.replace("’", "'")
    text = text.replace("‘", " ")
    text = text.replace("\t", " ")
    text = text.replace("!...", " ")
    text = text.replace("....", " ")
    text = text.replace("...", " ")
    text = text.replace(":", " ")
    text = text.replace(";", " ")
    text = text.replace("!", " ")
    text = text.replace(",", " ")
    text = text.replace("?", " ")
    text = text.replace("(", " ")
    text = text.replace(")", " ")
    text = text.replace("—", " ")
    text = text.replace("\r\n", " ")
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = text.replace(".", " ")
    text = text.replace("&", " and ")
    text = text.replace(" 57E ", " fifty seven e ")
    text = text.replace(" 1st ", " first ")
    text = text.replace(" 6th ", " sixth ")
    text = text.replace(" 1895 ", " eighteen ninety five ")
    text = text.replace(" 1891", " eighteen ninety one")
    text = text.replace("1891", " eighteen ninety one")
    text = text.replace("65 ", " sixty five ")
    if text[-1] == ".":
        text = text[:-1]
    text = re.sub("  +", " ", text)
    return text.lower().strip()

In [76]:
clean = [cleaner(x) for x in sents]

In [78]:
with open("dubliners-clean.txt", "w") as outf:
    for line in clean:
        outf.write(line + "\n")

In [79]:
!apt install -y build-essential libboost-all-dev cmake zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libbz2-dev is already the newest version (1.0.8-5build1).
libbz2-dev set to manually installed.
liblzma-dev is already the newest version (5.2.5-2ubuntu1).
liblzma-dev set to manually installed.
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu9.2).
zlib1g-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [80]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 14161, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (328/328), done.
remote: Total 14161 (delta 162), reused 406 (delta 132), pack-reused 13687
Receiving objects: 100% (14161/14161), 5.91 MiB | 9.48 MiB/s, done.
Resolving deltas: 100% (8042/8042), done.


In [81]:
%cd kenlm

/content/kenlm


In [82]:
!mkdir build
%cd build
!cmake ..
!make -j 4

/content/kenlm/build
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.74.0/BoostConfig.cmake (found suitable version "1.74.0", minimum

In [83]:
%cd /content

/content


In [84]:
!find kenlm -name lmplz

kenlm/build/bin/lmplz


In [86]:
!/content/kenlm/build/bin/lmplz -o 5 < /content/dubliners-clean.txt > dubliners.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/dubliners-clean.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 68170 types 7521
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:90252 2:1062191552 3:1991609344 4:3186574592 5:4647088128
Statistics:
1 7521 D1=0.613663 D2=1.27413 D3+=1.35107
2 36763 D1=0.80197 D2=1.15443 D3+=1.52879
3 56953 D1=0.9043 D2=1.32639 D3+=1.55938
4 60067 D1=0.964639 D2=1.51858 D3+=1.79285
5 56906 D1=0.97755 D2=1.72265 D3+=1.46156
Memory estimate for binary LM:
type      kB
probing 4795 assuming -p 1.5
probing 5725 assuming -r models -p 1.5
trie    2209 without quantization
trie    1174 assuming -q 8 -b 8 quantization 
trie    2042 assuming -a 22 array pointer compression
trie    1007 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calcul